In [3]:
import pandas as pd
import numpy as np
import datetime

In [5]:
df_GAM = pd.read_csv('/Users/lauralayton/Downloads/Chamber_GAM_model_data 2.csv')
print(df_GAM)

df_gg = pd.read_csv('/Users/lauralayton/Downloads/greenhouse_gas.csv')
print(df_gg)

       chamber_ID       Date GS_Interval_fld  Hydroperiod_days_fld   LAB_ID  \
0               1  7/11/2012           Early                  54.5  13060.0   
1               1  7/17/2012            Late                  54.5  13289.0   
2               1  7/19/2012            Late                  54.5  13355.0   
3               1  7/20/2012            Late                  54.5  13399.0   
4               1  7/23/2012            Late                  54.5  13443.0   
...           ...        ...             ...                   ...      ...   
18798           5  5/22/2015           Early                   0.0  34216.0   
18799           5   6/4/2015           Early                   0.0  34485.0   
18800           5   7/1/2015           Early                   0.0  34977.0   
18801           5  7/16/2015           Early                   0.0  35223.0   
18802           5  7/30/2015            Late                   0.0    960.7   

      Land_Cover_fld  methane_flux_log_mg_m2_hr  ND

In [20]:
# breaking the date into day, month, and year for both dataframes

# Adding Day, Month, and Year columns to the data
import datetime 
# dates = df.iloc[:,1].values

def convert_GAM_time_data(date_time):
    format = '%m/%d/%Y'
    datetime_str = datetime.datetime.strptime(date_time, format)

    return datetime_str

def convert_gg_time_data(date_time):
    format = '%d%b%Y'
    datetime_str = datetime.datetime.strptime(date_time, format)

    return datetime_str



df_GAM['Date Object'] = pd.to_datetime(df_GAM['Date'])
df_GAM['Day'] = pd.to_datetime(df_GAM['Date']).dt.day
df_GAM['Month'] = pd.to_datetime(df_GAM['Date']).dt.month
df_GAM['Year'] = pd.to_datetime(df_GAM['Date']).dt.year

df_gg['Date Object'] = df_gg['DATE'].apply(convert_gg_time_data)
df_gg['Date Object'] = pd.to_datetime(df_gg['Date Object'])
df_gg['Day'] = pd.to_datetime(df_gg['Date Object']).dt.day
df_gg['Month'] = pd.to_datetime(df_gg['Date Object']).dt.month
df_gg['Year'] = pd.to_datetime(df_gg['Date Object']).dt.year

In [21]:
print(df_gg)

      WETLAND_ID       DATE   LAB_ID  TYPE  CHAMBER_ID  AIR_TEMP_C  \
0         BECK 5  10JUL2012  13016.0     1           1        31.4   
1         BECK 5  10JUL2012  13017.0     1           2        31.4   
2         BECK 5  10JUL2012  13019.0     1           4        31.4   
3         BECK 5  10JUL2012  13020.0     1           5        31.4   
4         BECK 5  10JUL2012  13021.0     1           6        31.4   
...          ...        ...      ...   ...         ...         ...   
52334  UNIT 3, N  17JUN2013  21212.0     1           1        22.0   
52335  UNIT 3, N  10JUL2013  22384.0     1           1        24.1   
52336  UNIT 3, N  20AUG2013  24049.0     1           1        32.7   
52337  UNIT 3, N  17SEP2013  25419.0     1           1        22.0   
52338  UNIT 3, N  18OCT2013  27289.0     1           1         7.0   

       SOIL_MOISTURE_PERCENT  SW_TEMP_C  WATER_DEPTH_CM  G_CO2_M2_HR  ...  \
0                       57.1    -9999.0            52.0     0.131963  ...   
1    

In [29]:
# make a binary column depending on if the water depth is 0
def bin_water_depth(depth):
    if depth > 0:
        return 1
    else: 
        return 0

In [115]:
# aggregate data in the greenhouse gas dataframe to determine hydroperiod 
# in particular, aggregate the info in the WATER_DEPTH_CM column

# need to do this based on specific combos of wetland ID, chamber ID, and year

df_gg['WATER_PRESENT'] = df_gg['WATER_DEPTH_CM'].apply(bin_water_depth)

# group the greenhouse_gas data by WETLAND_ID, CHAMBER_ID, and Year
grouped = df_gg.groupby(['WETLAND_ID', 'CHAMBER_ID', 'Year'])['WATER_PRESENT'].sum()

In [116]:
grouped

WETLAND_ID       CHAMBER_ID  Year
BECK 5           1           2012    10
                             2013     9
                             2014    13
                             2015    12
                             2016     1
                                     ..
UNIT 3, E PENIN  1           2012     7
                             2013     5
UNIT 3, N        1           2010     4
                             2012     7
                             2013     6
Name: WATER_PRESENT, Length: 3963, dtype: int64

In [117]:
grouped = df_gg.groupby(['WETLAND_ID', 'CHAMBER_ID', 'Year']).agg(hydroperiod = ('WATER_PRESENT', 'sum'))
grouped

hydroperiod
WETLAND_ID      CHAMBER_ID Year             
BECK 5          1          2012           10
                           2013            9
                           2014           13
                           2015           12
                           2016            1
...                                      ...
UNIT 3, E PENIN 1          2012            7
                           2013            5
UNIT 3, N       1          2010            4
                           2012            7
                           2013            6

[3963 rows x 1 columns]

In [119]:
# flatten the GroupBy object back into a dataframe

grouped.columns = [''.join(col) for col in grouped.columns]
grouped = grouped.reset_index()

# for index, row in grouped.iterrows():
#     print(row)

In [120]:
df_gg['Hydroperiod'] = 2

# fill in hydroperiod for each observation
for index, row in df_gg.iterrows():
    yr = row['Year']
    wetland_ID = row['WETLAND_ID']
    chamber_ID = row['CHAMBER_ID']

    hydroperiod = grouped[(grouped['WETLAND_ID'] == wetland_ID) & (grouped['CHAMBER_ID'] == chamber_ID) 
        & (grouped['Year'] == yr)]['hydroperiod']
    # print(yr, wetland_ID, chamber_ID, hydroperiod)
    # print(hydroperiod)
    # print(index)
    # print(df_gg.loc[index, 'Hydroperiod'])
    # print(hydroperiod)
    # print(type(hydroperiod))
    # print(len(hydroperiod))

    # for i in hydroperiod.index:
    #     print(i)
    # df_gg.loc[index]['Hydroperiod'] = hydroperiod
    # df_gg.loc[index, 'Hydroperiod'] = hydroperiod[0]
    df_gg.loc[index, 'Hydroperiod'] = hydroperiod[hydroperiod.index[0]]

    # print(df_gg.loc[index])

In [77]:
print(df_gg[(df_gg['G_CH4_M2_HR'] < 0) & (df_gg['G_CH4_M2_HR'] > -100)]['G_CH4_M2_HR'])

df_test = df_gg[(df_gg['G_CH4_M2_HR'] < 0) & (df_gg['G_CH4_M2_HR'] > -100)]

min(df_test['G_CH4_M2_HR'])

4       -0.000019
5       -0.000033
12      -0.000022
13      -0.000063
14      -0.000043
           ...   
52287   -0.000022
52290   -0.000121
52307   -0.000035
52309   -0.000129
52325   -0.000005
Name: G_CH4_M2_HR, Length: 21050, dtype: float64


-0.000796195

In [78]:
grouped_smaller = df_GAM.groupby(['wetland_ID', 'chamber_ID', 'Year'])


In [121]:
print(df_gg)

      WETLAND_ID       DATE   LAB_ID  TYPE  CHAMBER_ID  AIR_TEMP_C  \
0         BECK 5  10JUL2012  13016.0     1           1        31.4   
1         BECK 5  10JUL2012  13017.0     1           2        31.4   
2         BECK 5  10JUL2012  13019.0     1           4        31.4   
3         BECK 5  10JUL2012  13020.0     1           5        31.4   
4         BECK 5  10JUL2012  13021.0     1           6        31.4   
...          ...        ...      ...   ...         ...         ...   
52334  UNIT 3, N  17JUN2013  21212.0     1           1        22.0   
52335  UNIT 3, N  10JUL2013  22384.0     1           1        24.1   
52336  UNIT 3, N  20AUG2013  24049.0     1           1        32.7   
52337  UNIT 3, N  17SEP2013  25419.0     1           1        22.0   
52338  UNIT 3, N  18OCT2013  27289.0     1           1         7.0   

       SOIL_MOISTURE_PERCENT  SW_TEMP_C  WATER_DEPTH_CM  G_CO2_M2_HR  ...  \
0                       57.1    -9999.0            52.0     0.131963  ...   
1    

In [82]:
print(len(df_gg.groupby(['WETLAND_ID', 'CHAMBER_ID', 'Year'])))
print(len(df_GAM.groupby(['wetland_ID', 'chamber_ID', 'Year'])))

3963
1977


In [131]:
# make some kind of summary table...
# wetland ID, chamber ID, number of years data, number of datapoints and yes and no to what is there in bansal vs all

grouped_gg_ID = df_gg.groupby(['WETLAND_ID', 'CHAMBER_ID'])

grouped_bansal_ID = df_GAM.groupby(['wetland_ID', 'chamber_ID'])

count_included = 0

summary_tbl = pd.DataFrame(columns=['WETLAND_ID',
                                    'CHAMBER_ID',
                                    # this next column is just a sanity check
                                    'IN greenhouse_gas.csv?',
                                    'IN BANSAL DATA?',
                                    'NUM OF YRS OF DATA - gg',
                                    'NUM OF YRS OF DATA - BANSAL',
                                    'NUM TOTAL OBS - gg',
                                    'NUM TOTAL OBS - BANSAL'])

in_gg = True
in_bansal = False

for key, group in grouped_gg_ID:
    wetland_ID = key[0]
    chamber_ID = key[1]

    df_in_gg = df_gg[(df_gg['WETLAND_ID'] == wetland_ID) &
        (df_gg['CHAMBER_ID'] == chamber_ID)]

    df_in_bansal = df_GAM[(df_GAM['wetland_ID'] == wetland_ID) &
        (df_GAM['chamber_ID'] == chamber_ID)]

    obs_in_gg = len(df_in_gg)
    obs_in_bansal = len(df_in_bansal)

    if obs_in_gg > 0:
        in_gg = True
    else:
        in_gg = False

    if obs_in_bansal > 0:
        in_bansal = True
    else:
        in_bansal = False

    num_gg_yrs = df_in_gg['Year'].nunique()
    num_bansal_yrs = df_in_bansal['Year'].nunique()

    new_row = pd.Series({'WETLAND_ID': wetland_ID, 
                        'CHAMBER_ID': chamber_ID,
                        'IN greenhouse_gas.csv?': in_gg,
                        'IN BANSAL DATA?': in_bansal,
                        'NUM OF YRS OF DATA - gg': num_gg_yrs,
                        'NUM OF YRS OF DATA - BANSAL': num_bansal_yrs,
                        'NUM TOTAL OBS - gg': obs_in_gg,
                        'NUM TOTAL OBS - BANSAL': obs_in_bansal})

    summary_tbl = pd.concat([summary_tbl, new_row.to_frame().T], ignore_index=True)


# now iterate through the bansal data just in case for some reason there's data there that they conjured
for key, group in grouped_bansal_ID:
    wetland_ID = key[0]
    chamber_ID = key[1]

    df_in_gg = df_gg[(df_gg['WETLAND_ID'] == wetland_ID) &
        (df_gg['CHAMBER_ID'] == chamber_ID)]

    df_in_bansal = df_GAM[(df_GAM['wetland_ID'] == wetland_ID) &
        (df_GAM['chamber_ID'] == chamber_ID)]

    obs_in_gg = len(df_in_gg)
    obs_in_bansal = len(df_in_bansal)

    if obs_in_gg > 0:
        in_gg = True
        continue
    else:
        in_gg = False

    if obs_in_bansal > 0:
        in_bansal = True
    else:
        in_bansal = False

    num_gg_yrs = df_in_gg['Year'].nunique()
    num_bansal_yrs = df_in_bansal['Year'].nunique()

    
    new_row = pd.Series({'WETLAND_ID': wetland_ID, 
                        'CHAMBER_ID': chamber_ID,
                        'IN greenhouse_gas.csv?': in_gg,
                        'IN BANSAL DATA?': in_bansal,
                        'NUM OF YRS OF DATA - gg': num_gg_yrs,
                        'NUM OF YRS OF DATA - BANSAL': num_bansal_yrs,
                        'NUM TOTAL OBS - gg': obs_in_gg,
                        'NUM TOTAL OBS - BANSAL': obs_in_bansal})

    summary_tbl = pd.concat([summary_tbl, new_row.to_frame().T], ignore_index=True)

# and it seems they did not conjure up data


In [132]:
summary_tbl

,WETLAND_ID,CHAMBER_ID,IN greenhouse_gas.csv?,IN BANSAL DATA?,NUM OF YRS OF DATA - gg,NUM OF YRS OF DATA - BANSAL,NUM TOTAL OBS - gg,NUM TOTAL OBS - BANSAL
0,BECK 5,1,True,True,5,4,60,47
1,BECK 5,2,True,True,4,4,55,49
2,BECK 5,3,True,True,4,4,56,47
3,BECK 5,4,True,True,5,4,58,47
4,BECK 5,5,True,True,5,4,60,46
...,...,...,...,...,...,...,...,...
1413,"UNIT 2, SOUTH",1,True,False,1,0,1,0
1414,"UNIT 2, SW",1,True,False,2,0,3,0
1415,"UNIT 3, C DIKE",1,True,False,3,0,19,0
1416,"UNIT 3, E PENIN",1,True,False,3,0,19,0


In [135]:
summary_tbl.to_excel("original_vs_bansal_summary.xlsx")

In [136]:
sum(summary_tbl['IN BANSAL DATA?'])

698

In [139]:
count_in_bansal = 0
for index, row in summary_tbl.iterrows():
    if row['IN BANSAL DATA?']:
        count_in_bansal += 1

print(count_in_bansal)

698


In [145]:
# check if use the same NDVI for all chambers in a wetland; Bansal data
count_multiple_NDVI = 0
group_bansal_by_wetland = df_GAM.groupby(['wetland_ID', 'Date'])
total_groups = len(group_bansal_by_wetland)
count_NDVI_vals = [0,0,0,0,0,0,0,0,0,0]

for key, group in group_bansal_by_wetland:
    df_subset = df_GAM[(df_GAM['wetland_ID'] == key[0])
        & (df_GAM['Date'] == key[1])]

    NDVI_vals = df_subset['NDVI_rs'].nunique()
    if NDVI_vals > 1:
        count_multiple_NDVI += 1
    count_NDVI_vals[NDVI_vals] += 1

print(total_groups)
print(count_multiple_NDVI)

print(count_NDVI_vals)

4755
2818
[0, 1937, 2085, 589, 141, 3, 0, 0, 0, 0]


In [148]:
df_gg['WETLAND_ID'].unique()

array(['BECK 5', 'BECK 6', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
       'CL01', 'CL05', 'CL07', 'CL08', 'DU01', 'DU02', 'DU03', 'DU05',
       'DU06', 'DU07', 'DU08', 'DU09', 'DU10', 'DU11', 'DU12', 'DU13',
       'DU14', 'DU15', 'DU17', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7',
       'F8', 'GP01157', 'GP01158', 'GP01314', 'GP01315', 'GP01317',
       'GP01318', 'GP01323', 'GP01324', 'GP01325', 'GP01326', 'GP01601',
       'GP01602', 'GP01603', 'GP01604', 'GP01605', 'GP01606', 'GP01607',
       'GP01608', 'GP01609', 'GP01610', 'GP01611', 'GP01612', 'GP01613',
       'GP01615', 'GP01617', 'GP01618', 'GP01619', 'GP01621', 'GP01622',
       'GP01623', 'GP01624', 'GP01626', 'GP01627', 'GP01630', 'GP01631',
       'GP01633', 'GP01634', 'GP01635', 'GP01637', 'GP01638', 'GP01639',
       'GP09640', 'GP09641', 'GP09642', 'GP09643', 'GP09644', 'GP09645',
       'GP09646', 'GP09647', 'GP09648', 'GP09649', 'GP09650', 'GP09651',
       'GP09652', 'GP09653', 'GP09654', 'GP09655', 'GP0965

In [149]:
df_GAM['wetland_ID'].unique()

array(['BECK 5', 'BECK 6', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
       'DU01', 'DU02', 'DU03', 'DU05', 'DU06', 'DU07', 'DU08', 'DU09',
       'DU10', 'DU11', 'DU12', 'DU13', 'DU14', 'DU15', 'DU17', 'F1', 'F2',
       'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'GP01157', 'GP01158',
       'GP01314', 'GP01317', 'GP01318', 'GP01323', 'GP01325', 'GP01326',
       'GP01604', 'GP01605', 'GP01606', 'GP01607', 'GP01608', 'GP01609',
       'GP01610', 'GP01611', 'GP01612', 'GP01613', 'GP01615', 'GP01617',
       'GP01618', 'GP01619', 'GP01624', 'GP01626', 'GP01627', 'GP01630',
       'GP01631', 'GP01633', 'GP01634', 'GP01635', 'GP01637', 'GP01638',
       'GP01639', 'GP09644', 'GP09645', 'GP09646', 'GP09647', 'GP09648',
       'GP09649', 'GP09650', 'GP09651', 'GP09652', 'GP09653', 'GP09654',
       'GP09655', 'GP09656', 'GP09657', 'GP09658', 'GP09659', 'GP09664',
       'GP09665', 'GP09666', 'GP09667', 'GP09668', 'GP09669', 'GP09670',
       'GP09671', 'GP09672', 'GP09673', 'GP09674', 'GP096

In [152]:
for wetland in df_gg['WETLAND_ID'].unique():
    print(wetland, end = '; ')

    

BECK 5; BECK 6; C1; C2; C3; C4; C5; C6; C7; C8; CL01; CL05; CL07; CL08; DU01; DU02; DU03; DU05; DU06; DU07; DU08; DU09; DU10; DU11; DU12; DU13; DU14; DU15; DU17; F1; F2; F3; F4; F5; F6; F7; F8; GP01157; GP01158; GP01314; GP01315; GP01317; GP01318; GP01323; GP01324; GP01325; GP01326; GP01601; GP01602; GP01603; GP01604; GP01605; GP01606; GP01607; GP01608; GP01609; GP01610; GP01611; GP01612; GP01613; GP01615; GP01617; GP01618; GP01619; GP01621; GP01622; GP01623; GP01624; GP01626; GP01627; GP01630; GP01631; GP01633; GP01634; GP01635; GP01637; GP01638; GP01639; GP09640; GP09641; GP09642; GP09643; GP09644; GP09645; GP09646; GP09647; GP09648; GP09649; GP09650; GP09651; GP09652; GP09653; GP09654; GP09655; GP09656; GP09657; GP09658; GP09659; GP09660; GP09661; GP09662; GP09663; GP09664; GP09665; GP09666; GP09667; GP09668; GP09669; GP09670; GP09671; GP09672; GP09673; GP09674; GP09675; GP09676; GP09677; GP09678; GP09679; GP11006; GP11010; GP11012; GP11014; GP11017; GP11018; GP11430; GP11680; GP116